# Dame Sermonde - Image Generation Notebook
This notebook loads a base model and a LoRA adapter, then generates multiple image variations for each prompt defined in `config.json`.

In [1]:
import os
import json
import torch
from pathlib import Path
from diffusers import FluxPipeline
from huggingface_hub import login
from PIL import Image


/workspace/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 🔐 Load Hugging Face token
HF_TOKEN = os.getenv("HUGGINGFACE_HUB_TOKEN")
if not HF_TOKEN:
    try:
        with open("secrets/hf_token.txt") as f:
            HF_TOKEN = f.read().strip()
            os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    except FileNotFoundError:
        raise ValueError("No Hugging Face token found. Set HUGGINGFACE_HUB_TOKEN or create secrets/hf_token.txt")
login(token=HF_TOKEN)


# 🔧 Load FLUX pipeline

In [ ]:
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    torch_dtype=torch.float16,
    use_auth_token=True
).to("cuda")


# 📄 Load config.json

In [ ]:
with open("config.json", "r") as f:
    config = json.load(f)

lora = config["lora"]
pipe.load_lora_weights(
    lora["lora_repo"],
    weight_name=lora["weight_name"],
    adapter_name=lora["adapter_name"]
)
pipe.set_adapters([lora["adapter_name"]])
print(f"✅ Loaded LoRA: {lora['adapter_name']}")


# 🎨 Generate images

## 1️⃣ Single image

In [ ]:
output_dir = "outputs/test"
prompt = """DameSermonde sitting at a carved wooden vanity table, applying medieval makeup with a calm yet mildly annoyed expression, wearing a silk underdress, soft candlelight, gold-trimmed mirror, small glass jars and brushes on the table, elegant castle chamber background, Instagram-ready"""

In [ ]:
weight = 0.7
base_seed = 42
num_variations = 8

output_dir.mkdir(parents=True, exist_ok=True)

with open(output_dir / "prompt.txt", "w", encoding="utf-8") as f:
    f.write(prompt)

for j in range(num_variations):
    seed = base_seed + j
    generator = torch.manual_seed(seed)
    pipe.set_adapters([lora["adapter_name"]], adapter_weights=[weight])
    image = pipe(prompt=prompt, generator=generator).images[0]

    filename = f"damesermonde_w{int(weight*100)}_s{seed}.png"
    image.save(output_dir / filename)

    print(f"✅ Saved: {output_dir / filename}")

## 🔢 Multiple images (from config)

In [ ]:
weight = 0.7
base_seed = 42
num_variations = 8

for i, prompt_cfg in enumerate(config["prompts"]):
    prompt = prompt_cfg["prompt"]
    output_dir = Path(prompt_cfg["output_dir"])
    output_dir.mkdir(parents=True, exist_ok=True)

    with open(output_dir / "prompt.txt", "w", encoding="utf-8") as f:
        f.write(prompt)

    for j in range(num_variations):
        seed = base_seed + j
        generator = torch.manual_seed(seed)
        pipe.set_adapters([lora["adapter_name"]], adapter_weights=[weight])
        image = pipe(prompt=prompt, generator=generator).images[0]

        filename = f"damesermonde_w{int(weight*100)}_s{seed}.png"
        image.save(output_dir / filename)

        print(f"✅ Saved: {output_dir / filename}")


# 🖼️ Display images

In [ ]:
from IPython.display import display, HTML
from PIL import Image
import os
from pathlib import Path
import base64
from io import BytesIO

def display_images_grid(directory, max_per_row=3, image_width=200):
    directory = Path(directory)
    image_tags = []

    for image_file in sorted(directory.glob("*.png")):
        with Image.open(image_file) as img:
            buffer = BytesIO()
            img.save(buffer, format="PNG")
            img_b64 = base64.b64encode(buffer.getvalue()).decode("utf-8")
            tag = f"""
            <div style="margin: 10px; text-align: center;">
                <img src="data:image/png;base64,{img_b64}" style="width: {image_width}px;"><br>
                <span style="font-size: 12px;">{image_file.name}</span>
            </div>
            """
            image_tags.append(tag)

    # Group by row
    rows = [
        image_tags[i:i + max_per_row]
        for i in range(0, len(image_tags), max_per_row)
    ]

    html = ""
    for row in rows:
        html += f"<div style='display: flex; flex-direction: row; flex-wrap: nowrap;'>{''.join(row)}</div>"

    display(HTML(html))

# 🔍 Example usage
display_images_grid("outputs/prompt_001")